In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import ndcg_score
import math

In [ ]:
# Read data
shopee = pd.read_csv('/content/gdrive/MyDrive/Hệ khuyến nghị/kem_info.csv')
shopee=shopee.drop(labels='Unnamed: 0',axis=1)
shopee=shopee.rename(columns = {'name':'product_name'})

In [ ]:
shopee[shopee['description']=='Null']

## Xử lí dữ liệu, loại bỏ null

In [ ]:
shopee=shopee[shopee['avg_rating']!="Null"]

In [ ]:
shopee['avg_rating'] = shopee['avg_rating'].astype(float)
shopee=shopee[shopee['avg_rating']>=4.0]

<ipython-input-6-cd918a6be54b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shopee['avg_rating'] = shopee['avg_rating'].astype(float)


In [ ]:
shopee.dtypes

product_id        int64
product_name     object
brand            object
description      object
price            object
avg_rating      float64
luot_mua         object
dtype: object

In [ ]:
shopee['price'].isnull().sum()
shopee['price'].fillna('-1', inplace=True)

In [ ]:
price_list = []
shopee['price'].fillna('-1', inplace=True)
for i in shopee['price'].str.split(" - "):
  
  if len(i)==1:
    price_list.append(int(''.join(i[0].split('.'))))
  elif len(i)==2:
    price_list.append(int(''.join(i[1][1:].split('.'))))
  
shopee['price'] = price_list

In [ ]:
# shopee=shopee.drop(labels='luot_mua',axis=1)
# shopee=shopee.drop(labels='price',axis=1)
# shopee=shopee.drop(labels='avg_rating',axis=1)
shopee=shopee.drop(labels='brand',axis=1)


In [ ]:
shopee.head(2)

,product_id,product_name,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",Null,399000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,Null,419000,4.9,"10,9kđã bán"


In [ ]:
shopee.drop_duplicates(subset=['product_name'])

,product_id,product_name,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",Null,399000,5.0,2kđã bán
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,Null,419000,4.9,"10,9kđã bán"
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158000,4.9,"101,5kđã bán"
3,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,Thời gian giao hàng dự kiến cho sản phẩm này l...,56000,4.9,"106,2kđã bán"
4,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,ALT Beauty\nLưu ý: sản phẩm được sản xuất tại ...,135000,5.0,50kđã bán
...,...,...,...,...,...,...
295,12461191193,Kem chống nắng Omi Sun Bears Strong Super Plus...,Null,59000,4.9,323đã bán
296,14991138669,Kem Chống Nắng K Lady Care Sun Cream 50ml,Null,248000,4.8,11đã bán
297,9048731252,Kem Chống Nắng Cell Fusion C,Null,380000,5.0,311đã bán
298,15010825241,Kem Chống Nắng Fixderma Shadow 75g,Null,241000,5.0,204đã bán


In [ ]:
shopee['luot_mua'] = shopee['luot_mua'].apply(lambda x: x[0:-6])
def process_luotmua(row):
  if 'k' in row and ',' in row:
    return int(row[0:-3])*1000+int(row[-2])*100
  elif 'k' in row:
    return int(row[0:-1])*1000
  else:
    return int(row)
shopee['luot_mua'] = shopee['luot_mua'].apply(process_luotmua)
shopee.sort_values(by=['luot_mua'],ascending=False)

,product_id,product_name,description,price,avg_rating,luot_mua
3,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,Thời gian giao hàng dự kiến cho sản phẩm này l...,56000,4.9,106200
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158000,4.9,101500
63,2258622483,(Đủ Bill) Kem chống nắng Cell Fusion C Laser /...,INBOX CHAT VỚI SHOP ĐỂ XEM HÌNH SP VÀ BILL MUA...,305000,5.0,54500
4,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,ALT Beauty\nLưu ý: sản phẩm được sản xuất tại ...,135000,5.0,50000
34,18539937099,Kem chống nắng lâu trôi làm sáng da innisfree ...,ALT Beauty\nThông Tin Sản Phẩm\nFREESHIP-Kem c...,129000,5.0,50000
...,...,...,...,...,...,...
206,13990420779,Kem chống nắng vật lý nâng tone Peacholic - Hà...,Kem chống nắng vật lý nâng tone Peacholic - Hà...,155000,5.0,5
235,17545615416,Kem chống nắng AMPLE:N Hyaluronshot Suncare SP...,Shop cam kết tất cả các sản phẩm của shop đều ...,184000,5.0,4
205,17525751684,Kem chống nắng chính hãng lâu trôi làm trắng d...,Kem chống nắng chính hãng lâu trôi Innisfree S...,199000,5.0,4
228,18845483508,Kem Chống Nắng Shiseido Spf50 + + + 100ml Bảo ...,Null,204000,5.0,3


In [ ]:
df=shopee

## TF-IDF

In [ ]:
df=shopee[['product_id', 'product_name', 'description', 'luot_mua']]
# df.drop_duplicates(inplace=True)
df=df[df['description']!='Null']


In [ ]:
index_dup = df['product_name'].drop_duplicates().index.tolist()

In [ ]:
df = df.loc[index_dup, :]
df.reset_index(drop=True, inplace=True)

In [ ]:
df['luot_mua'] = df['luot_mua']/df['luot_mua'].max()

In [ ]:
df['luot_mua']= df['luot_mua']/df['luot_mua'].max()

In [ ]:
# df.to_csv('/content/gdrive/MyDrive/Hệ khuyến nghị/itembasecsv')

In [ ]:
# Break up the big genre string into a string array
df['description'] = df['description'].str.split('|')
# Convert genres to string value
df['description'] = df['description'].fillna("").astype('str')

In [ ]:
def preprocessing(text):
  text = text.lower()

  html = re.compile("<.*?>")
  text = html.sub(r"", text)

  text = re.sub("[^\w\s]", "", text)
  text = re.sub('\n','',text)

  return text

In [ ]:
process_discription = []
# Recommend dựa trên description
for i in df['description'].values:
  process_discription.append(preprocessing(i))

In [ ]:
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(process_discription)
tfidf_matrix

<207x5648 sparse matrix of type '<class 'numpy.float64'>'
	with 41017 stored elements in Compressed Sparse Row format>

In [ ]:

cosine_sim = linear_kernel(tfidf_matrix.toarray(), tfidf_matrix.toarray())

In [ ]:
df['product_id']

0       5386949803
1       3240946968
2      21239933571
3      11506865254
4      18558394870
          ...     
202    19824983620
203    12125389908
204    19047195116
205      165944581
206     4586385199
Name: product_id, Length: 207, dtype: int64

In [ ]:
df

,product_id,product_name,description,luot_mua
0,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"['""Vaseline tin rằng bất cứ ai cũng xứng đáng ...",0.955744
1,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,['Thời gian giao hàng dự kiến cho sản phẩm này...,1.000000
2,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,['ALT Beauty\nLưu ý: sản phẩm được sản xuất tạ...,0.470810
3,11506865254,(MẪU MỚI) Kem Chống Nắng Nâng Tone da Innisfre...,[' Kem Chống Nắng Nâng Tone da Innisfree Tone ...,0.035782
4,18558394870,Kem Chống Nắng PrettySkin - Pretty Skin Sun Cr...,['Kem Chống Nắng Pretty Skin Sun Cream SPF 50+...,0.000443
...,...,...,...,...
202,19824983620,Kem chống nắng nâng tone ma:nyo Foundation-Fre...,['✅HƯỚNG DẪN SỬ DỤNG✅ Trong bước cuối cùng của...,0.011299
203,12125389908,[MẪU MỚI 2022] Kem chống nắng cho da dầu EUCER...,['Amélie _ Dược mỹ phẩm Pháp 🇫🇷\nAmélie cam kế...,0.000471
204,19047195116,Kem Chống Nắng La Roche Posay dành cho da nhạy...,['✅ Vie Beauté Đại Lý Phân Phối Chính Thức Dượ...,0.000254
205,165944581,KEM CHỐNG NẮNG INNISFREE DAILY UV PROTECTION C...,['KEM CHỐNG NẮNG INNISFREE DAILY \nUV PROTECTI...,0.015066


## Mô hình cho người dùng chưa có dữ liệu

In [ ]:
shopee.dtypes

product_id        int64
product_name     object
description      object
price             int64
avg_rating      float64
luot_mua          int64
dtype: object

In [ ]:
shopee

,product_id,product_name,description,price,avg_rating,luot_mua
0,19762237765,"Kem Chống Nắng Cho Da Dầu, Mụn, Nám Và Tăng Sắ...",Null,399000,5.0,2000
1,17061960863,Sữa chống nắng bảo vệ da tối ưu khỏi tia UVA d...,Null,419000,4.9,10900
2,5386949803,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...,"""Vaseline tin rằng bất cứ ai cũng xứng đáng sở...",158000,4.9,101500
3,3240946968,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...,Thời gian giao hàng dự kiến cho sản phẩm này l...,56000,4.9,106200
4,21239933571,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...,ALT Beauty\nLưu ý: sản phẩm được sản xuất tại ...,135000,5.0,50000
...,...,...,...,...,...,...
295,12461191193,Kem chống nắng Omi Sun Bears Strong Super Plus...,Null,59000,4.9,323
296,14991138669,Kem Chống Nắng K Lady Care Sun Cream 50ml,Null,248000,4.8,11
297,9048731252,Kem Chống Nắng Cell Fusion C,Null,380000,5.0,311
298,15010825241,Kem Chống Nắng Fixderma Shadow 75g,Null,241000,5.0,204


In [ ]:
shopee=shopee.sort_values(by=['luot_mua'],ascending=False).iloc[:20]
shopee=shopee[['product_name']]
shopee

,product_name
172,Kem chống nắng nâng tông da SOME BY MI Truecic...
12,Kem Chống nắng SJM Medical Anti UV Suncream 60g
37,Kem chống nắng ngừa mụn Caryophy Smart Sunscre...
253,[Hàng mới về] /////// AHC Natural Perfection ...
192,Kem chống nắng cho da nhạy cảm Blue Lizard Sen...
81,Kem chống nắng La Roche posay anthelios
14,Kem Chống Nắng SJM Chống Nước Nâng Tone Da Dàn...
199,Sữa chống nắng Skin Aqua SPF 50+. Kem dưỡng tr...
148,Kem chống nắng Cell Fusion C Toning Suncreen 1...
223,[New 2021] Kem Chống Nắng Innisfree TRIPLE SHE...


In [ ]:
np.array(shopee)

array([['Kem chống nắng nâng tông da SOME BY MI Truecica Mineral Calming Tone-up Suncream SPF50+ PA++++ 50ml'],
       ['Kem Chống nắng SJM Medical Anti UV Suncream 60g'],
       ['Kem chống nắng ngừa mụn Caryophy Smart Sunscreen 50ml và 5ml ILIVING-CARKCN'],
       ['[Hàng mới về] ///////  AHC Natural Perfection Double Shield Sun Stick 14g, 22g /////// Kem chống nắng AHC dạng thỏi bảo vệ gấp 2 lần 14g 22g / TOPKOREA'],
       ['Kem chống nắng cho da nhạy cảm Blue Lizard Sensitive Sunscreen SPF50+'],
       ['Kem chống nắng La Roche posay anthelios'],
       ['Kem Chống Nắng SJM Chống Nước Nâng Tone Da Dành Cho Da Dầu Mụn, Da Nhạy Cảm, Da Khô Hàn Quốc Chính Hãng 60ml CN05'],
       ['Sữa chống nắng Skin Aqua SPF 50+. Kem dưỡng trắng da ban ngày lên tone tự nhiên.'],
       ['Kem chống nắng Cell Fusion C Toning Suncreen 100 .dung tích 50ml . An toàn và dễ sử dụng'],
       ['[New 2021] Kem Chống Nắng Innisfree TRIPLE SHEILD/ LONG LASTING Lâu Trôi Kềm Dầu SPF50PA++++'],
       ['Kem chốn

In [ ]:
shopee.index

Int64Index([3, 2, 63, 4, 34, 60, 83, 217, 80, 161, 41, 58, 43, 154, 23, 115, 7,
            44, 52, 36],
           dtype='int64')

In [ ]:
groundtruth = df_user[df_user['username'].isin(df_us['username'].values)][['username', 'product_name']].drop_duplicates()
groundtruth.index = groundtruth['username']
groundtruth.drop(['username'], axis=1, inplace=True)

In [ ]:
df_pre=pd.DataFrame(columns = ['user_name','product_pre'])

list_user=[]
list_pro = []
for key in user_dict.keys():
    list_user = list_user + [key for i in range(20)]
for username in df_us.username:
  for product_name in shopee.product_name:
    list_pro.append(product_name)

df_pre['user_name'] = list_user
df_pre['product_pre'] = list_pro


In [ ]:
df_pre

,user_name,product_pre
0,p*****g,Kem Lót Trang Điểm FOCALLURE Kiềm Dầu Nâng Tôn...
1,p*****g,Serum chống nắng cơ thể Vaseline 50x bảo vệ da...
2,p*****g,(Đủ Bill) Kem chống nắng Cell Fusion C Laser /...
3,p*****g,Kem chống nắng kiềm dầu nâng tông Sunplay Skin...
4,p*****g,Kem chống nắng lâu trôi làm sáng da innisfree ...
...,...,...
395,m*****i,[Mẫu mới] Kem chống nắng La RochePosay Antheli...
396,m*****i,[Bạn mới nhập COSHBM -50% đơn 0Đ] Kem chống ti...
397,m*****i,Mẫu mới kem Chống Nắng The SAEM Eco Earth Powe...
398,m*****i,Kem chống nắng Năng động Cancer Council Active...


### PtopK

In [ ]:
def PtopK(username):
  count = 0
  for m in df_pre[df_pre['user_name']==username].product_pre:
    for s in groundtruth[groundtruth.index==username].product_name:
      if(m == s):
        count+=1
  PtopK = count/len(df_pre[df_pre['user_name']==username])
  return PtopK

In [ ]:
sum_ptopk=0
for username in df_us['username']:
  sum_ptopk += PtopK(username)

print("mean PtopK of 20 users is: ", sum_ptopk/len(df_us))


### MRR

In [ ]:
def get_pred_user(username, df_pre, groundtruth):
  MRR_predict = []
  pred = df_pre[df_pre['user_name']==username]['product_pre'].values.tolist()
  true = groundtruth.loc[username, 'product_name'].values.tolist()
  for i in pred:
    if i in true:
      MRR_predict.append(1)
    else:
      MRR_predict.append(0)
  return MRR_predict


In [ ]:
def RRu_score(MRR_predict):
    RRu = 0
    for i in range(len(MRR_predict)):
      if MRR_predict[i] == 1:
        rr = 1/(i+1)
        RRu+=rr
        break;
    # print(RRu)
    return RRu

In [ ]:
def MRR(ListUser):
  MRR = 0
  
  for user in ListUser:
      MRR_predict = get_pred_user(user, df_pre, groundtruth)
      RRu = RRu_score(MRR_predict)
      MRR += RRu
      print("MRRu score for user {} : {}".format(user, RRu))
  return (1/len(ListUser))*MRR

In [ ]:
ListUser = df_us['username'].values.tolist()

MRR_score = MRR(ListUser)

In [ ]:
print(MRR_score)

### NDCG score

In [ ]:
def DCG_score(predict_ranking):
    DCG = 0
    for i in range(len(predict_ranking)):
        DCG += predict_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def iDCG_score(true_ranking):
    DCG = 0
    for i in range(len(true_ranking)):
        DCG += true_ranking[i]/math.log2(i+1+1)
    return DCG

In [ ]:
def NDCG_for_listUser(ListUser):
    ndcg = 0
    for user in ListUser:
        
        prec_ranking = get_pred_user(user, df_pre, groundtruth)
        true_ranking = sorted(prec_ranking, reverse=True)
        print("predict_ranking")
        print(prec_ranking)
        print("true_ranking")
        print(true_ranking)
        dcg = DCG_score(prec_ranking)
        idcg = iDCG_score(true_ranking)
        try:
          ndcg += dcg/idcg
          print("ndcg for user {}: {}".format(user, dcg/idcg))
        except:
          ndcg +=0
        
    return (1/len(ListUser))*ndcg

In [ ]:
df_pre.iloc[0:20]['product_pre']

In [ ]:
groundtruth.iloc[40:60]['product_name']

In [ ]:
ListUser = df_us['username'].values.tolist()

NDCG_score = NDCG_for_listUser(ListUser)

In [ ]:
NDCG_score